In [1]:
###R preprocessing Code
##reutersImportantCSV <- reutersCSV[,c("pid","fileName","purpose","topic.earn",\
##"topic.acq", "topic.money.fx","topic.grain", "topic.crude", "topic.trade", "topic.interest",\
##"topic.ship","topic.wheat","topic.corn","doc.title","doc.text")]

##reutersNUImportantCSV <- reutersImportantCSV[which(reutersImportantCSV$purpose=='not-used'),]

##reutersTestImportantCSV <- reutersImportantCSV[which(reutersImportantCSV$purpose=='test'),]

##reutersTrainImportantCSV <- reutersImportantCSV[which(reutersImportantCSV$purpose=='train'),]

##write.csv(reutersNUImportantCSV, file = "reutersNUImportantCSV.csv",row.names=TRUE, na="")

##write.csv(reutersTestImportantCSV, file = "reutersTestImportantCSV.csv",row.names=TRUE, na="")

##write.csv(reutersTrainImportantCSV, file = "reutersTrainImportantCSV.csv",row.names=TRUE, na="")

##write.csv(reutersImportantCSV, file = "reutersImportantCSV.csv",row.names=TRUE, na="")

import os
import pandas as pd

# Sets/changes the directory to the document directory
os.chdir("/Users/Akhilesh Pandey/Desktop/art_corpus/ART_Corpus")

#Reading the Training file and saving into data frame
reutersTrainDataSet=pd.read_csv("reutersTrainImportantCSV.csv",encoding='iso-8859-1',sep=',')

In [2]:
#We will combine the topics and the text of each row and create an entry
#in the class List as the class for which the value is '1'

classList=[]
textList=[]
textCombination=[]
textCombination = reutersTrainDataSet['doc.title'].map(str) + ' '+ reutersTrainDataSet['doc.text']
#Looking for condition '1'. Also flag remains 'False' if all the 10 topics have 0 value
for i in range(0, len(reutersTrainDataSet)):
    flag= False
    if reutersTrainDataSet['topic.earn'][i]==1:
        classList.append('topic.earn')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.money.fx'][i]==1:
        classList.append('topic.money.fx')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.acq'][i]==1:
        classList.append('topic.acq')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.grain'][i]==1:
        classList.append('topic.grain')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.crude'][i]==1:
        classList.append('topic.crude')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.trade'][i]==1:
        classList.append('topic.trade')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.interest'][i]==1:
        classList.append('topic.interest')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.ship'][i]==1:
        classList.append('topic.ship')
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.wheat'][i]==1:
        classList.append('topic.wheat')        
        textList.append(textCombination[i])
        flag = True
    if reutersTrainDataSet['topic.corn'][i]==1:
        classList.append('topic.corn')
        textList.append(textCombination[i])
        flag = True
    if flag is False:
        classList.append('No Class')
        textList.append(textCombination[i])
print(len(classList))
print(len(textList))
print(len(reutersTrainDataSet))

15373
15373
14668


In [3]:
import nltk 
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

lmtzr = WordNetLemmatizer()
st = LancasterStemmer()

# Stores the stopwords in english in a set called stop
stop=set(stopwords.words('english'))

def process_words(words):
    
    # Initialising the list to store the stop words
    wordList= []
    wordLemmaList= []
    wordStemList = []
    
    # Converts all the uppercase in the Tweets to lowercase
    words=words.lower()
    
    #Removing the special symbols such as ',','.', etc.
    words = re.sub(r'[^a-zA-Z0-9 ]',r'',words)
    
    #Converting stream of words to a list of words to check for stop words
    wordList= words.split()
    
    #Performing Lemmatisation
    for words in wordList:
        wordLemmaList.append(lmtzr.lemmatize(words))
        
    #Performing Word Stemming
    for words in wordList:        
        wordStemList.append(st.stem(words))
    
    #Removing the stopwords from the List of words and joins the words into string
    for words in wordStemList:
        if words in stop:
            wordStemList.remove(words)
    return(" ".join(wordStemList))

clean_text=[]

num_rows= len(textList)
#Iterating through each row and proessing the text.
#We're saving this output in clean_text List
for i in range(0, num_rows):
    clean_text.append(process_words(str(textList[i])))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialising the Vectoriser based on processed Tweets and taking max_features 1000
vectorizer = CountVectorizer(analyzer = "word",ngram_range=(1,3), max_features=1500, stop_words = None) 

#Creating the vectorizer for clean_text List 
#Also we are fitting the vectorizer 
text_features = vectorizer.fit_transform(clean_text)

# Converts the feature matrix to an array
text_features = text_features.toarray()

print(type(text_features))

<class 'numpy.ndarray'>


In [5]:
#Creating a K-fold function
from sklearn.metrics import accuracy_score
import numpy as np

text_features_list = text_features.tolist()
#The function splits the featureList and the classList based on the 'folds'
#The output will be 4 lists. The devided feature List and divided class List
def k_fold_generator(featureList, classList, folds):    
    subset_size = int(len(featureList) / folds)  
    for k in range(folds):
        featureList_train = featureList[:k * subset_size] + featureList[(k + 1) * subset_size:]
        featureList_test = featureList[k * subset_size:][:subset_size]
        classList_train = classList[:k * subset_size] + classList[(k + 1) * subset_size:]
        classList_test = classList[k * subset_size:][:subset_size]
        yield featureList_train, classList_train, featureList_test, classList_test

In [6]:
#Testing on Naive Bayes
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
#We will use the function created above and use it to create 10 seaprate sub sections of the data set
#While the 9 of the sections will be used for training, the other 10th will be used for testing
#We will iterate 10 times until we have covered all the subsections as test dataset
#the .score function gives the accuracy of the prediction
text_features_list = text_features.tolist()         
for featureList_train, classList_train, featureList_test, classList_test in k_fold_generator(text_features_list,classList,10):
    featureList_train = np.asarray(featureList_train)
    classList_train = np.asarray(classList_train)
    mnb.fit(featureList_train,classList_train)  #The .score method produces 
    score = mnb.score(featureList_test,classList_test)
    print(score)

0.635653871178
0.66753415745
0.646063760573
0.666883539362
0.666883539362
0.661027976578
0.635653871178
0.62589459987
0.642160052049
0.446324007807


In [7]:
#Testing on Random Forest 
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 20)
#We are doing the same thing for the Random Forest Classifier. The parameters havebeen discussed
#in the report.
text_features_list = text_features.tolist()        
for featureList_train, classList_train, featureList_test, classList_test in k_fold_generator(text_features_list,classList,10):
    featureList_train = np.asarray(featureList_train)
    classList_train = np.asarray(classList_train)
    forest.fit(featureList_train,classList_train)
    score = forest.score(featureList_test, classList_test)
    print(score)

0.793754066363
0.797657774886
0.750813272609
0.788549121666
0.805465191932
0.75992192583
0.780091086532
0.753415744958
0.737150292778
0.867273910215


In [8]:
#Testng on SVM
from sklearn import svm
import numpy as np 
svm = svm.SVC(kernel='rbf') 

#We are using the 'rbf' kernel as this it is the most efficient kernel for 
#text classification

text_features_list = text_features.tolist()   
for featureList_train, classList_train, featureList_test, classList_test in k_fold_generator(text_features_list,classList,10):
    featureList_train = np.asarray(featureList_train)
    classList_train = np.asarray(classList_train)
    svm.fit(featureList_train,classList_train)
    score = svm.score(featureList_test, classList_test)
    print(score)

0.79440468445
0.793103448276
0.754716981132
0.790500975927
0.802212101496
0.763825634353
0.776837996096
0.74821080026
0.730644111906
0.880936890046


In [9]:
#Reading the Test file 
reutersTestDataSet=pd.read_csv("reutersTestImportantCSV.csv",encoding='iso-8859-1',sep=',')
#Doing test test processing similar to train data set
classTestList=[]
textTestList=[]
textCombination=[]
textCombination = reutersTestDataSet['doc.title'].map(str) + ' '+ reutersTrainDataSet['doc.text']
for i in range(0, len(reutersTestDataSet)):
    flag= False
    if reutersTestDataSet['topic.earn'][i]==1:
        classTestList.append('topic.earn')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.money.fx'][i]==1:
        classTestList.append('topic.money.fx')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.acq'][i]==1:
        classTestList.append('topic.acq')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.grain'][i]==1:
        classTestList.append('topic.grain')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.crude'][i]==1:
        classTestList.append('topic.crude')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.trade'][i]==1:
        classTestList.append('topic.trade')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.interest'][i]==1:
        classTestList.append('topic.interest')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.ship'][i]==1:
        classTestList.append('topic.ship')
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.wheat'][i]==1:
        classTestList.append('topic.wheat')        
        textTestList.append(textCombination[i])
        flag = True
    if reutersTestDataSet['topic.corn'][i]==1:
        classTestList.append('topic.corn')
        textTestList.append(textCombination[i])
        flag = True
    if flag is False:
        classTestList.append('No Class')
        textTestList.append(textCombination[i])

clean_test_text=[]

num_rows= len(textTestList)

for i in range(0, num_rows):
    clean_test_text.append(process_words(str(textList[i])))

test_data_features = vectorizer.transform(clean_test_text)

test_data_features = test_data_features.toarray()

In [10]:
#Training Random Forest 
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 20) 
predictRFModel = forest.fit(text_features,classList)

#Implementing Random Forest
resultRF = predictRFModel.predict(test_data_features)

In [11]:
#Training the Naive Bayes Model
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
predictNBModel = mnb.fit(text_features,classList)

#Implementing Naive Bayes
resultNB = predictNBModel.predict(test_data_features)

In [12]:
#Training the SVM Model
from sklearn import svm
import numpy as np 
svm = svm.SVC(kernel='rbf') 
predictSVMModel = svm.fit(text_features,classList)

#Implementing SVM
resultSVM = predictSVMModel.predict(test_data_features)

In [13]:
#Creating the Confusion Matrix Random Forest
from nltk.metrics import ConfusionMatrix 
import numpy as np

trueValues =  classTestList
calculatedValues = resultRF.tolist()
confusionMatrixRF = ConfusionMatrix(trueValues, calculatedValues)
print(confusionMatrixRF)

               |                                  t    t                |
               |                                  o    o                |
               |                                  p    p                |
               |                   t         t    i    i         t    t |
               |              t    o    t    o    c    c    t    o    o |
               |         t    o    p    o    p    .    .    o    p    p |
               |    N    o    p    i    p    i    i    m    p    i    i |
               |    o    p    i    c    i    c    n    o    i    c    c |
               |         i    c    .    c    .    t    n    c    .    . |
               |    C    c    .    c    .    g    e    e    .    t    w |
               |    l    .    c    r    e    r    r    y    s    r    h |
               |    a    a    o    u    a    a    e    .    h    a    e |
               |    s    c    r    d    r    i    s    f    i    d    a |
               |    s    q    n    e  

In [14]:
#Creating the Confusion Matrix Naive Bayes
from nltk.metrics import ConfusionMatrix 
import numpy as np

trueValues =  classTestList
calculatedValues = resultNB.tolist()
confusionMatrixNB = ConfusionMatrix(trueValues, calculatedValues)
print(confusionMatrixNB)

               |                                  t    t                |
               |                                  o    o                |
               |                                  p    p                |
               |                   t         t    i    i         t    t |
               |              t    o    t    o    c    c    t    o    o |
               |         t    o    p    o    p    .    .    o    p    p |
               |    N    o    p    i    p    i    i    m    p    i    i |
               |    o    p    i    c    i    c    n    o    i    c    c |
               |         i    c    .    c    .    t    n    c    .    . |
               |    C    c    .    c    .    g    e    e    .    t    w |
               |    l    .    c    r    e    r    r    y    s    r    h |
               |    a    a    o    u    a    a    e    .    h    a    e |
               |    s    c    r    d    r    i    s    f    i    d    a |
               |    s    q    n    e  

In [15]:
#Creating the Confusion Matrix SVM
from nltk.metrics import ConfusionMatrix 
import numpy as np

trueValues =  classTestList
calculatedValues = resultSVM.tolist()
confusionMatrixSVM = ConfusionMatrix(trueValues, calculatedValues)
print(confusionMatrixSVM)

               |                                  t    t                |
               |                                  o    o                |
               |                                  p    p                |
               |                   t         t    i    i         t    t |
               |              t    o    t    o    c    c    t    o    o |
               |         t    o    p    o    p    .    .    o    p    p |
               |    N    o    p    i    p    i    i    m    p    i    i |
               |    o    p    i    c    i    c    n    o    i    c    c |
               |         i    c    .    c    .    t    n    c    .    . |
               |    C    c    .    c    .    g    e    e    .    t    w |
               |    l    .    c    r    e    r    r    y    s    r    h |
               |    a    a    o    u    a    a    e    .    h    a    e |
               |    s    c    r    d    r    i    s    f    i    d    a |
               |    s    q    n    e  

In [16]:
#Function to calculate Stats 
from collections import Counter

def CalculateStats(confusionMatrix):
    labels = ['No Class','topic.acq','topic.corn','topic.crude','topic.earn','topic.grain','topic.interest',
              'topic.money.fx','topic.ship','topic.trade']
    true_positives = Counter()
    false_negatives = Counter()
    false_positives = Counter()

    for i in labels:
        for j in labels:
            if i == j:
                true_positives[i] += confusionMatrix[i,j]
            else:
                false_negatives[i] += confusionMatrix[i,j]
                false_positives[j] += confusionMatrix[i,j]

    for i in sorted(labels):
        if true_positives[i] == 0:
            fscore = 0
        else:
            precision = true_positives[i] / float(true_positives[i]+false_positives[i])
            recall = true_positives[i] / float(true_positives[i]+false_negatives[i])
            fscore = 2 * (precision * recall) / float(precision + recall)
        print (i, fscore)

In [17]:
CalculateStats(confusionMatrixRF)

No Class 0.5662718556119571
topic.acq 0.10795040116703135
topic.corn 0
topic.crude 0.028089887640449437
topic.earn 0.18458859381802348
topic.grain 0.04129793510324484
topic.interest 0.02531645569620253
topic.money.fx 0.029239766081871343
topic.ship 0.023255813953488372
topic.trade 0.024291497975708502


In [18]:
CalculateStats(confusionMatrixNB)

No Class 0.44253269098417064
topic.acq 0.1614481409001957
topic.corn 0.017699115044247787
topic.crude 0.035532994923857864
topic.earn 0.1705357142857143
topic.grain 0.013605442176870748
topic.interest 0.026246719160104987
topic.money.fx 0.03194888178913738
topic.ship 0.027303754266211604
topic.trade 0.02364066193853428


In [19]:
CalculateStats(confusionMatrixSVM)

No Class 0.5997904662126767
topic.acq 0.1012448132780083
topic.corn 0
topic.crude 0.024539877300613494
topic.earn 0.17425227568270482
topic.grain 0.038356164383561646
topic.interest 0.011627906976744186
topic.money.fx 0.0303951367781155
topic.ship 0
topic.trade 0.018433179723502308
